In [22]:
# Imports
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType


import json
import os

import mysql.connector
from enum import Enum

import pyspark.sql.functions as func
import typing

import schema_patterns


In [23]:

def append_JSON_obects(filename: str) -> list[str]:
    """
    Returns all JSON objects in file appended to a single list

    Parameters:
        filename (str): The name of the file containing the JSON objects
    
    Returns:
        branch_list (list[str]): The list of json obects
    
    """
    branch_list = []

    with open(filename) as f:
        data = json.load(f)

    for i in data:
        branch_list.append(i)

    f.close()

    return branch_list



In [24]:
def extract_from_directory(directoryName: str) -> list[str]:
    """
    Returns a dictionary of JSON file names within the given directory

    Parameters: 
        directoryName (str): the name of a directory containing only JSON files

    Returns:
        parsed_list (list[str]): list of file names within the directory
    """
    parsed_list = {}
    for filename in os.listdir(directoryName):
        f = os.path.join(directoryName, filename)
        if os.path.isfile(f):
            table_name = filename.split(".")[0].upper()
            parsed_list[table_name] = append_JSON_obects(f)
        else:
            print("Unable to find", f)
    return parsed_list

In [25]:
class TBL_NAME(Enum):
    CREDIT = "CDW_SAPP_CREDIT"
    BRANCH = "CDW_SAPP_BRANCH"
    CUST = "CDW_SAPP_CUSTOMER"

The customer data table not have an area code. It appears almost every phone number in the branch database uses 123 - this appears to be a placeholder value, so I'm just applying it to the customer table as well.

## STrecth goal: IMPORT DATASET FOR AREA CODE ZIP CODE/ADDRESS TO PUT REAL VALUE

In [26]:
# create the SparkSession
spark = SparkSession.builder.appName('Bank_Analysis').getOrCreate()

parsed_lists = extract_from_directory("json")
df_dict = {}
for name in parsed_lists:
    df_dict[name] = spark.createDataFrame(parsed_lists[name])


for k,v in df_dict.items():
    print(k,v)
    df_dict[k].createOrReplaceTempView(k)
    temp = spark.sql(schema_patterns.pattern_dict[k])
    df_dict[k] = temp

   


CDW_SAPP_CREDIT DataFrame[BRANCH_CODE: bigint, CREDIT_CARD_NO: string, CUST_SSN: bigint, DAY: bigint, MONTH: bigint, TRANSACTION_ID: bigint, TRANSACTION_TYPE: string, TRANSACTION_VALUE: double, YEAR: bigint]
CDW_SAPP_BRANCH DataFrame[BRANCH_CITY: string, BRANCH_CODE: bigint, BRANCH_NAME: string, BRANCH_PHONE: string, BRANCH_STATE: string, BRANCH_STREET: string, BRANCH_ZIP: bigint, LAST_UPDATED: string]
CDW_SAPP_CUSTOMER DataFrame[APT_NO: string, CREDIT_CARD_NO: string, CUST_CITY: string, CUST_COUNTRY: string, CUST_EMAIL: string, CUST_PHONE: bigint, CUST_STATE: string, CUST_ZIP: string, FIRST_NAME: string, LAST_NAME: string, LAST_UPDATED: string, MIDDLE_NAME: string, SSN: bigint, STREET_NAME: string]


NameError: name 'rdd_customer' is not defined

24/06/14 16:56:59 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
